## Setup and Import Libraries

In [ ]:
import os
import numpy as np
from typing import List
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [17]:
load_dotenv()

True

In [18]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [19]:
llm = ChatOpenAI(model='gpt-3.5-turbo')
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
groq_llm = init_chat_model(model="groq:openai/gpt-oss-20b")

## Data Ingestion and Processing

In [5]:
sample_documents = [
    Document(
        page_content="""
        Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
        """,
        metadata={"source": "AI Introduction", "page": 1, "topic": "AI"}
    ),
    Document(
        page_content="""
        Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.
        """,
        metadata={"source": "ML Basics", "page": 1, "topic": "ML"}
    ),
    Document(
        page_content="""
        Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
        """,
        metadata={"source": "Deep Learning", "page": 1, "topic": "DL"}
    ),
    Document(
        page_content="""
        Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
        Applications include chatbots, translation, sentiment analysis, and text summarization.
        """,
        metadata={"source": "NLP Overview", "page": 1, "topic": "NLP"}
    )
]

print(sample_documents)

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='\n        Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.\n        '), Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='\n        Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.\n        '), Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='\n        Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolu

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[" "],
    chunk_size=500,
    chunk_overlap=50,
    length_function=len)

chunks = text_splitter.split_documents(documents=sample_documents)

In [7]:
print(chunks[0])
print()
print(chunks[1])

page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.' metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}

page_content='Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.' metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}


In [8]:
print(f"Created {len(chunks)} chunks from {len(sample_documents)} documents")
print("\nExample chunk:")
print(f"Content: {chunks[0].page_content}")
print(f"Metadata: {chunks[0].metadata}")

Created 4 chunks from 4 documents

Example chunk:
Content: Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
Metadata: {'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}


## Create Vector Store

In [9]:
vector_store = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

print(f"Vector store created with {vector_store.index.ntotal} vectors")

vector_store

Vector store created with 4 vectors


### Saving Vector Store

In [10]:
vector_store.save_local("faiss_index")

print("Vector store saved to 'faiss_index' directory")

Vector store saved to 'faiss_index' directory


### Loading Vector Store

In [11]:
loaded_vector_store = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

print(f"Loaded vector store contains {loaded_vector_store.index.ntotal} vectors")

Loaded vector store contains 4 vectors


## Perform Similarity Search

In [12]:
query = "What os Deep Learning"

results = loaded_vector_store.similarity_search(query=query, k=3)
print(results)

[Document(id='f350d08e-7b13-4c16-834e-dfc103a3a8a0', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recognition.'), Document(id='4bf0d25e-5e34-407d-ae14-e9b76562a874', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'), Document(id='713c8d02-2c42-4198-85bb-5aa7363efec9', metadata={'source': 'NLP Overview', 'page': 1, 'topic': 'NLP'}, page_content='Natural Language Processing (NLP) is a branch of AI that helps computers understand human lang

In [13]:
print(f"Query: {query}")
print("Top 3 Similar Chunks")

for i, result in enumerate(results):
    print(f"\n{i + 1}. Source: {result.metadata['source']}")
    print(f"    Content: {result.page_content[:200]}...") 

Query: What os Deep Learning
Top 3 Similar Chunks

1. Source: Deep Learning
    Content: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning ...

2. Source: ML Basics
    Content: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised...

3. Source: NLP Overview
    Content: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
      ...


In [14]:
query = "What os Deep Learning"

results_with_scores = loaded_vector_store.similarity_search_with_score(query=query, k=3)

print(f"Query: {query}")
print("Top 3 Similar Chunks with Score")

for result, score in results_with_scores:
    print(f"Score: {float(score):.3f}")
    print(f"Source: {result.metadata.get("source", "N/A")}")
    print(f"Content preview: {result.page_content[:100]}...\n")

Query: What os Deep Learning
Top 3 Similar Chunks with Score
Score: 0.606
Source: Deep Learning
Content preview: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses m...

Score: 1.194
Source: ML Basics
Content preview: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being...

Score: 1.278
Source: NLP Overview
Content preview: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
...



### Search with Metadata Filtering

In [15]:
filter_dict = {"topic": "ML"}

filtered_results = loaded_vector_store.similarity_search(
    query=query, 
    k=3, 
    filter=filter_dict
)

print(filtered_results)

[Document(id='4bf0d25e-5e34-407d-ae14-e9b76562a874', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.')]


## Build RAG Chain with LCEL

In [20]:
simple_prompt = ChatPromptTemplate.from_template(""" 
Answer the question based only on the following context:
Context: {context}
                                                 
Question: {question}

Answer:                                                       
""")

In [21]:
retriever = loaded_vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [ ]:
def format_documents(documents: List[Document]) -> str:
    """
    Format documents for insertion into prompt
    """

    formatted_document = []
    for i, document in enumerate(documents):
        source = document.metadata.get('source', 'Unknown')
        formatted_document.append(f"Document {i+1} (Source: {source}):\n{document.page_content}")

    return "\n\n".join(formatted_document)

In [23]:
simple_rag_chain = (
    {
        "context": retriever | format_documents,
        "question": RunnablePassthrough()
    }
    | simple_prompt
    | groq_llm
    | StrOutputParser()
)

simple_rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E38ABE72F0>, search_kwargs={'k': 3})
           | RunnableLambda(format_documents),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=' \nAnswer the question based only on the following context:\nContext: {context}\n\nQuestion: {question}\n\nAnswer:                                                       \n'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E3896C7050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E38A8A6060>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutput

## Conversational RAG Chain

In [24]:
conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer questions."),
    ("placeholder", "{chat_history}"),
    ("human", "Context: {context}\n\nQuestion: {input}"),
])

In [25]:
def create_conversational_rag():
    """Create a conversational RAG chain with memory"""
    return (
        RunnablePassthrough.assign(
            context=lambda x: format_documents(retriever.invoke(x["input"]))
        )
        | conversational_prompt
        | llm
        | StrOutputParser()
    )

conversational_rag = create_conversational_rag()

conversational_rag

RunnableAssign(mapper={
  context: RunnableLambda(lambda x: format_documents(retriever.invoke(x['input'])))
})
| ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag

## Streaming RAG Chain

In [26]:
streaming_rag_chain = (
    {
        "context": retriever | format_documents, 
        "question": RunnablePassthrough()
    }
    | simple_prompt
    | llm
)

print("Modern RAG chains created successfully!")

Modern RAG chains created successfully!


### Available chains:
- Simple_rag_chain: Basic Q&A"
- Conversational_rag: Maintains conversation history
- Streaming_rag_chain: Supports token streaming

## Test RAG Chains

In [27]:
def test_rag_chains(question: str):
    """
    Test all RAG chain variants
    """

    print(f"Question: {question}")
    print("=" * 80)

    # 1. Simple RAG
    print("\n1. Simple RAG Chain:")
    answer = simple_rag_chain.invoke(question)
    print(f"Answer: {answer}")

    # 1. Streaming RAG
    print("\n2. Streaming RAG:")
    print("Answer: ", end="", flush=True)
    for chunk in streaming_rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print()

In [28]:
question = "What is the difference between AI and machine learning"

test_rag_chains(question=question)

Question: What is the difference between AI and machine learning

1. Simple RAG Chain:
Answer: AI is the broader field that seeks to simulate human intelligence in machines—enabling systems to think, reason, and act like humans.  
Machine learning is a specific subset of AI that focuses on algorithms that learn patterns from data rather than being explicitly programmed. Thus, all machine‑learning systems are AI, but not all AI systems use machine learning.

2. Streaming RAG:
Answer: AI is the simulation of human intelligence in machines, while machine learning is a subset of AI that enables systems to learn from data. AI focuses on mimicking human actions and thinking, while machine learning focuses on finding patterns in data without being explicitly programmed.


In [29]:
test_questions = [
    "What is the difference between AI and Machine Learning?",
    "Explain deep learning in simple terms",
    "How does NLP work?"
]

for question in test_questions:
    print("\n" + "=" * 80 + "\n")
    test_rag_chains(question)



Question: What is the difference between AI and Machine Learning?

1. Simple RAG Chain:
Answer: **Answer:**  
Artificial Intelligence (AI) is the broader field that simulates human intelligence in machines, enabling them to think and act like humans. Machine Learning (ML) is a subset of AI that focuses on enabling systems to learn from data by discovering patterns, rather than being explicitly programmed. Thus, ML is one approach within the larger domain of AI.

2. Streaming RAG:
Answer: AI is the simulation of human intelligence in machines, while Machine Learning is a subset of AI that enables systems to learn from data. AI is designed to think like humans and mimic their actions, while Machine Learning algorithms find patterns in data instead of being explicitly programmed.


Question: Explain deep learning in simple terms

1. Simple RAG Chain:
Answer: Deep learning is a kind of machine learning that uses artificial neural networks with many layers.  
These layers work together to

In [30]:
print("\n3. Conversational RAG Example:")
chat_history = []

# First question
question1 = "What is machine learning?"
answer1 = conversational_rag.invoke({
    "input": question1,
    "chat_history": chat_history
})

print(f"Question 1: {question1}")
print(f"Answer 1: {answer1}")


3. Conversational RAG Example:
Question 1: What is machine learning?
Answer 1: Machine Learning is a subset of AI that enables systems to learn from data without being explicitly programmed. It involves using algorithms to find patterns in data, and common types include supervised, unsupervised, and reinforcement learning.


In [31]:
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

In [34]:
question2 = "How is it different from traditional programming?"

answer2 = conversational_rag.invoke({
    "input": question2,
    "chat_history": chat_history
})

print(f"\nQuestion 2: {question2}")
print(f"Answer 2: {answer2}")


Question 2: How is it different from traditional programming?
Answer 2: Traditional programming involves writing explicit instructions for a computer to follow, while machine learning involves training algorithms to learn from data and make predictions or decisions based on that data. In traditional programming, the programmer specifies the rules and logic for the computer to execute, whereas in machine learning, the system learns from examples and data to improve its performance over time.
